# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f33bca59400>
├── 'a' --> tensor([[ 0.1150,  1.1463, -0.1647],
│                   [-0.6285, -0.8272, -0.6268]])
└── 'x' --> <FastTreeValue 0x7f33bca59c70>
    └── 'c' --> tensor([[ 1.5618,  1.9370, -0.3951,  0.9192],
                        [-1.2633,  2.2905,  0.8027,  0.7669],
                        [ 2.2391, -0.6602, -0.3649, -0.0025]])

In [4]:
t.a

tensor([[ 0.1150,  1.1463, -0.1647],
        [-0.6285, -0.8272, -0.6268]])

In [5]:
%timeit t.a

59.3 ns ± 0.0136 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f33bca59400>
├── 'a' --> tensor([[-0.1088, -1.6247, -0.1649],
│                   [-0.1775,  1.9818,  0.0354]])
└── 'x' --> <FastTreeValue 0x7f33bca59c70>
    └── 'c' --> tensor([[ 1.5618,  1.9370, -0.3951,  0.9192],
                        [-1.2633,  2.2905,  0.8027,  0.7669],
                        [ 2.2391, -0.6602, -0.3649, -0.0025]])

In [7]:
%timeit t.a = new_value

61.9 ns ± 0.0159 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1150,  1.1463, -0.1647],
               [-0.6285, -0.8272, -0.6268]]),
    x: Batch(
           c: tensor([[ 1.5618,  1.9370, -0.3951,  0.9192],
                      [-1.2633,  2.2905,  0.8027,  0.7669],
                      [ 2.2391, -0.6602, -0.3649, -0.0025]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1150,  1.1463, -0.1647],
        [-0.6285, -0.8272, -0.6268]])

In [11]:
%timeit b.a

53.7 ns ± 0.0337 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.9286, -1.1807, -1.1510],
               [ 0.1098,  1.8201, -0.0817]]),
    x: Batch(
           c: tensor([[ 1.5618,  1.9370, -0.3951,  0.9192],
                      [-1.2633,  2.2905,  0.8027,  0.7669],
                      [ 2.2391, -0.6602, -0.3649, -0.0025]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.124 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

824 ns ± 0.108 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 26.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 452 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3306d06d30>
├── 'a' --> tensor([[[ 0.1150,  1.1463, -0.1647],
│                    [-0.6285, -0.8272, -0.6268]],
│           
│                   [[ 0.1150,  1.1463, -0.1647],
│                    [-0.6285, -0.8272, -0.6268]],
│           
│                   [[ 0.1150,  1.1463, -0.1647],
│                    [-0.6285, -0.8272, -0.6268]],
│           
│                   [[ 0.1150,  1.1463, -0.1647],
│                    [-0.6285, -0.8272, -0.6268]],
│           
│                   [[ 0.1150,  1.1463, -0.1647],
│                    [-0.6285, -0.8272, -0.6268]],
│           
│                   [[ 0.1150,  1.1463, -0.1647],
│                    [-0.6285, -0.8272, -0.6268]],
│           
│                   [[ 0.1150,  1.1463, -0.1647],
│                    [-0.6285, -0.8272, -0.6268]],
│           
│                   [[ 0.1150,  1.1463, -0.1647],
│                    [-0.6285, -0.8272, -0.6268]]])
└── 'x' --> <FastTreeValue 0x7f3306d06c40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.7 µs ± 97 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f330dd043a0>
├── 'a' --> tensor([[ 0.1150,  1.1463, -0.1647],
│                   [-0.6285, -0.8272, -0.6268],
│                   [ 0.1150,  1.1463, -0.1647],
│                   [-0.6285, -0.8272, -0.6268],
│                   [ 0.1150,  1.1463, -0.1647],
│                   [-0.6285, -0.8272, -0.6268],
│                   [ 0.1150,  1.1463, -0.1647],
│                   [-0.6285, -0.8272, -0.6268],
│                   [ 0.1150,  1.1463, -0.1647],
│                   [-0.6285, -0.8272, -0.6268],
│                   [ 0.1150,  1.1463, -0.1647],
│                   [-0.6285, -0.8272, -0.6268],
│                   [ 0.1150,  1.1463, -0.1647],
│                   [-0.6285, -0.8272, -0.6268],
│                   [ 0.1150,  1.1463, -0.1647],
│                   [-0.6285, -0.8272, -0.6268]])
└── 'x' --> <FastTreeValue 0x7f330738db20>
    └── 'c' --> tensor([[ 1.5618,  1.9370, -0.3951,  0.9192],
                        [-1.2633,  2.2905,  0.8027,  0.7669],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 67.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.7 µs ± 68.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.5618,  1.9370, -0.3951,  0.9192],
                       [-1.2633,  2.2905,  0.8027,  0.7669],
                       [ 2.2391, -0.6602, -0.3649, -0.0025]],
              
                      [[ 1.5618,  1.9370, -0.3951,  0.9192],
                       [-1.2633,  2.2905,  0.8027,  0.7669],
                       [ 2.2391, -0.6602, -0.3649, -0.0025]],
              
                      [[ 1.5618,  1.9370, -0.3951,  0.9192],
                       [-1.2633,  2.2905,  0.8027,  0.7669],
                       [ 2.2391, -0.6602, -0.3649, -0.0025]],
              
                      [[ 1.5618,  1.9370, -0.3951,  0.9192],
                       [-1.2633,  2.2905,  0.8027,  0.7669],
                       [ 2.2391, -0.6602, -0.3649, -0.0025]],
              
                      [[ 1.5618,  1.9370, -0.3951,  0.9192],
                       [-1.2633,  2.2905,  0.8027,  0.7669],
                       [ 2.2391, -0.6602, -0.3649, -0.0025]],

In [26]:
%timeit Batch.stack(batches)

77.3 µs ± 85.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.5618,  1.9370, -0.3951,  0.9192],
                      [-1.2633,  2.2905,  0.8027,  0.7669],
                      [ 2.2391, -0.6602, -0.3649, -0.0025],
                      [ 1.5618,  1.9370, -0.3951,  0.9192],
                      [-1.2633,  2.2905,  0.8027,  0.7669],
                      [ 2.2391, -0.6602, -0.3649, -0.0025],
                      [ 1.5618,  1.9370, -0.3951,  0.9192],
                      [-1.2633,  2.2905,  0.8027,  0.7669],
                      [ 2.2391, -0.6602, -0.3649, -0.0025],
                      [ 1.5618,  1.9370, -0.3951,  0.9192],
                      [-1.2633,  2.2905,  0.8027,  0.7669],
                      [ 2.2391, -0.6602, -0.3649, -0.0025],
                      [ 1.5618,  1.9370, -0.3951,  0.9192],
                      [-1.2633,  2.2905,  0.8027,  0.7669],
                      [ 2.2391, -0.6602, -0.3649, -0.0025],
                      [ 1.5618,  1.9370, -0.3951,  0.9192],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 171 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 1.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
